**Creation of an SQLite Database in python**

*Import necessary library*

In [27]:
import pandas as pd
from sqlite3 import connect
connection = connect('worm_genome.sqlite')

cursor = connection.cursor()



""


Creation of a feature table 

In [136]:
sql = '''
CREATE TABLE IF NOT EXISTS features (
        feature_id INTEGER PRIMARY KEY AUTOINCREMENT,
        seq_id TEXT,
        source TEXT,
        type TEXT,
        start INTEGER,
        end INTEGER,
        score TEXT,
        strand TEXT,
        phase TEXT
    );
'''
try:
    cursor.execute(sql)
except connection.DatabaseError:
    print("Creating the features table resulted in a database error!")
    connection.rollback()
    raise
else:
    connection.commit()
finally:
    print("done!")

done!


Checking to see what tables i have in my database

In [141]:
sql = '''
SELECT name,type 
FROM sqlite_master;
'''

cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
res_df

,0,1
0,sqlite_sequence,table
1,attributes,table
2,features,table
3,type_idx,index
4,start_idx,index
5,end_idx,index


Creation of an attributes table for my database

In [129]:
sql = '''
CREATE TABLE IF NOT EXISTS attributes (
        attr_id INTEGER PRIMARY KEY AUTOINCREMENT,
        feature_id INTEGER,
        attr_name TEXT,
        value TEXT,
        FOREIGN KEY (feature_id) REFERENCES features(feature_id)
    );
'''
try:
    cursor.execute(sql)
except connection.DatabaseError:
    print("Creating the features attributes table resulted in a database error!")
    connection.rollback()
    raise
else:
    connection.commit()
finally:
    print("done!")

done!


checks

In [130]:
sql = '''
SELECT name,type 
FROM sqlite_master;
'''

cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
res_df

,0,1
0,features,table
1,sqlite_sequence,table
2,attributes,table


Creation of indexes for the feature table on the type, start, and end columns

In [139]:
sql1 = '''
CREATE INDEX IF NOT EXISTS type_idx 
ON features (type);
'''
sql2 = '''
CREATE INDEX IF NOT EXISTS start_idx 
ON features (start);
'''
sql3 = '''
CREATE INDEX IF NOT EXISTS end_idx 
ON features (end);
'''
cursor.execute(sql1)
cursor.execute(sql2)
cursor.execute(sql3)
connection.commit()

Creation of indexes for the attributes table on the feature_id and name columns

In [142]:
sql1 = '''
CREATE INDEX IF NOT EXISTS feature_id_idx 
ON attributes (feature_id);
'''
sql2 = '''
CREATE INDEX IF NOT EXISTS attr_name_idx 
ON attributes (attr_name);
'''
cursor.execute(sql1)
cursor.execute(sql2)
connection.commit()

Checks

In [143]:
sql = '''
SELECT name,type 
FROM sqlite_master;
'''

cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
res_df

,0,1
0,sqlite_sequence,table
1,attributes,table
2,features,table
3,type_idx,index
4,start_idx,index
5,end_idx,index
6,feature_id_idx,index
7,attr_name_idx,index


Creation of a function for parsing the worm genome GFF3 file. 
    Population of the database after parsing the file.

In [146]:
def parse_gff3_line(line):
    parts = line.strip().split('\t')
    if len(parts) < 9:
        return None  # Not enough data in this line
    seq_id, source, feature_type, start, end, score, strand, phase, attributes = parts[0], parts[1], parts[2], parts[3], parts[4], parts[5], parts[6], parts[7], parts[8]
    
    # Extracting the name from the attributes
    attributes_dict = dict(field.split('=', 1) for field in attributes.split(';') if '=' in field)
    
    return seq_id, source, feature_type, start, end, score, strand, phase, attributes_dict

In [149]:
with open('worm_genome.gff3', 'r') as file:
    for line in file:
        if line.startswith('#'):
            continue  # Skip comment lines
        parsed_line = parse_gff3_line(line)
        if parsed_line is not None:
            cursor.execute("INSERT INTO features (seq_id, source, type, start, end, score, strand, phase) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", parsed_line[0:8])
            feature_id = cursor.lastrowid  # Get the generated feature_id

            # Insert into attributes table
            if parsed_line[8] is not None:
                attributes_dict = parsed_line[8]
                for attr_name, value in attributes_dict.items():
                    cursor.execute('''
                        INSERT INTO attributes (feature_id, attr_name, value)
                        VALUES (?, ?, ?)
                    ''', (feature_id, attr_name, value))

connection.commit()


Visualizing the Features table.

In [151]:
sql = '''
SELECT * 
FROM features;
'''

cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
header = [row[0] for row in cursor.description]
res_df.columns = header
res_df


,feature_id,seq_id,source,type,start,end,score,strand,phase
0,1,I,WormBase,chromosome,1,15072434,.,.,.
1,2,I,WormBase,chromosome,1,15072434,.,.,.
2,3,I,WormBase,ncRNA_gene,3747,3909,.,-,.
3,4,I,WormBase,snoRNA,3747,3909,.,-,.
4,5,I,WormBase,exon,3747,3909,.,-,.
...,...,...,...,...,...,...,...,...,...
659617,659618,X,WormBase,exon,17717328,17717443,.,+,.
659618,659619,X,WormBase,CDS,17717328,17717443,.,+,0
659619,659620,X,WormBase,CDS,17718435,17718621,.,+,1
659620,659621,X,WormBase,exon,17718435,17718720,.,+,.


Visualizing the attributes table

In [150]:
sql = '''
SELECT * 
FROM attributes;
'''

cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
header = [row[0] for row in cursor.description]
res_df.columns = header
res_df

,attr_id,feature_id,attr_name,value
0,1,2,ID,chromosome:I
1,2,2,Alias,"BX284601.5,NC_003279.8"
2,3,3,ID,gene:WBGene00023193
3,4,3,Name,Y74C9A.6
4,5,3,biotype,snoRNA
...,...,...,...,...
3436217,3436218,659621,ensembl_phase,2
3436218,3436219,659621,exon_id,cTel55X.1a.e5
3436219,3436220,659621,rank,5
3436220,3436221,659621,version,1


SQL Query that retrieves the number of rows in the features table.

In [153]:
sql = '''
SELECT COUNT(*) 
FROM features;
'''
cursor.execute(sql)
row_count = cursor.fetchall()
print(f"There are {row_count[0][0]} rows in the table features")

There are 659622 rows in the table features


SQL Query that retrieves the number of rows in the attributes table.

In [154]:
sql = '''
SELECT COUNT(*) 
FROM attributes;
'''
cursor.execute(sql)
row_count = cursor.fetchall()
print(f"There are {row_count[0][0]} rows in the table attributes")

There are 3436222 rows in the table attributes


SQL Query that displays the type and the number of features for each type from
the features table

In [155]:
sql = '''
SELECT type, COUNT(*) as num_features
FROM features
GROUP BY type;
'''
cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
header = [row[0] for row in cursor.description]
res_df.columns = header
res_df

,type,num_features
0,CDS,222859
1,chromosome,8
2,exon,269904
3,five_prime_UTR,30960
4,gene,20222
5,lnc_RNA,283
6,mRNA,33391
7,miRNA,454
8,ncRNA,8402
9,ncRNA_gene,24765


SQL Query that displays the number of genes on chromosome 1

In [160]:
sql = '''
SELECT COUNT(*)
FROM features
WHERE seq_id = 'I' AND type = 'gene';
'''
cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
header = [row[0] for row in cursor.description]
res_df.columns = header
res_df

,COUNT(*)
0,2889


SQL Query that displays the seq_id and type from the features table and the attr_name
and value from the attributes table for all features with the type chromosome

In [163]:
sql = '''
SELECT f.seq_id, f.type, a.attr_name, a.value
FROM features f
JOIN attributes a ON f.feature_id = a.feature_id
WHERE f.type = 'chromosome';
'''

cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
header = [row[0] for row in cursor.description]
res_df.columns = header
res_df

,seq_id,type,attr_name,value
0,I,chromosome,ID,chromosome:I
1,I,chromosome,Alias,"BX284601.5,NC_003279.8"
2,II,chromosome,ID,chromosome:II
3,II,chromosome,Alias,"BX284602.5,NC_003280.10"
4,III,chromosome,ID,chromosome:III
5,III,chromosome,Alias,"BX284603.4,NC_003281.10"
6,IV,chromosome,ID,chromosome:IV
7,IV,chromosome,Alias,"BX284604.4,NC_003282.8"
8,MtDNA,chromosome,ID,chromosome:MtDNA
9,MtDNA,chromosome,Alias,"X54252.1,NC_001328.1"


SQL Query that displays the type from the features table and the number of attributes
(using also the attributes table) for each feature type that has more than 40 attributes

In [164]:
sql = '''
SELECT f.type, COUNT(a.attr_id) as num_attributes
FROM features f
JOIN attributes a ON f.feature_id = a.feature_id
GROUP BY f.type
HAVING COUNT(a.attr_id) > 40;
'''

cursor.execute(sql)
res_data = cursor.fetchall()
res_df = pd.DataFrame(res_data)
header = [row[0] for row in cursor.description]
res_df.columns = header
res_df

,type,num_attributes
0,CDS,668577
1,exon,2159232
2,five_prime_UTR,30960
3,gene,111530
4,lnc_RNA,1415
5,mRNA,166955
6,miRNA,2270
7,ncRNA,42010
8,ncRNA_gene,123825
9,piRNA,76820
